In [1]:
# - 売り上げ高
# - 営業利益
# まで
# かなりめんどくさい…

# todo 
# 営業活動キャッシュフロー
# 営業活動キャッシュフロー/売上=営業キャッシュフローマージン計算
# 営業キャッシュフローマージンをendDatetimeで降順ソートして直近5年(出せるとこまで、2年が限界か？)
# 算出し、15%以上が2年以上ならOperating_cash_flow_margin_listに銘柄を追加

In [2]:
import glob
import os

from arelle.ModelValue import qname
from arelle import Cntlr
import numpy as np

import pandas as pd
pd.reset_option('display.max_rows')
print(pd.options.display.max_rows)
pd.reset_option('display.max_columns')
print(pd.options.display.max_columns)
import plotly.graph_objects as go

import stock_research as sr
taiouhyou = sr.taiouhyou


60
20


In [3]:
stock_code_list_path = '../warehouse/stock/created/submitted_doc_list_only_stock*.csv'
files = glob.glob(stock_code_list_path)
for file in files:
    print(file)

../warehouse/stock/created\submitted_doc_list_only_stock_20220101_20220428.csv
../warehouse/stock/created\submitted_doc_list_only_stock_20220301_20220428.csv
../warehouse/stock/created\submitted_doc_list_only_stock_20220330_20220428.csv


In [4]:
orig_data_path = '../warehouse/stock/original_data/'
stock_code_list = '../warehouse/stock/created/submitted_doc_list_only_stock_20220101_20220428.csv'
code_df = pd.read_csv(stock_code_list)

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
stock_code

6697

In [31]:
file_path

'../warehouse/stock/original_data/2022-03-11/S100NN9G/XBRL/PublicDoc/*.xbrl'

In [35]:
j=-1
# for stock_code in code_df['secCode'].unique():
# for stock_code in np.sort(code_df['secCode'].unique()):
# for i,stock_code in enumerate([1720]):
# for i, stock_code in enumerate([6025]):
for i, stock_code in enumerate(np.sort(code_df['secCode'].unique())):
    doc_id = code_df.query(f'secCode=={stock_code}').docID.values[0]
    date = code_df.query(f'secCode=={stock_code}').submitDateTime.values[0]
    filerName = code_df.query(f'secCode=={stock_code}').filerName.values[0]
    class_33 = code_df.query(f'secCode=={stock_code}')['33業種区分'].values[0]
    class_17 = code_df.query(f'secCode=={stock_code}')['17業種区分'].values[0]

    file_path = orig_data_path + date[:10] + '/' + doc_id + '/XBRL/PublicDoc/*.xbrl'
    try:
        file_path = glob.glob(file_path)[0]
    except:
        continue
    fact_df = sr.read_xbrl(file_path,stock_code)
    
    mask = fact_df.contextID.str.contains('NonConsolidated')
    mask = mask.astype(int)
    fact_df['NonConsolidated'] = mask
    
    # プロットに向かない数値以外のデータをすべて除外
    mask = fact_df['value'].str.match('^[0-9]+$')
    fact_df = fact_df[mask]
    
    for k,key in enumerate(taiouhyou):
        tmp = sr.get_key(fact_df,key)
        
        if k==0:
            sdf = tmp
        else:
            sdf = pd.concat([sdf,tmp],axis=0)
            
    sdf = sr.make_operatingincome_margin(sdf)
    sdf['filerName'] = filerName
    sdf['secCode'] = stock_code
    sdf['33業種区分'] = class_33
    sdf['17業種区分'] = class_17

    
    # print(sdf.shape)
            
    if i==0:
        df = sdf
    else:
        df = pd.concat([df,sdf],axis=0)
    
    if i%50==0:print(i,'/',len(code_df['secCode'].unique()))


0 / 686
50 / 686
100 / 686
150 / 686
200 / 686
250 / 686
300 / 686
350 / 686
400 / 686
450 / 686
500 / 686
550 / 686
600 / 686
650 / 686


In [36]:
df = sr.save_stock_statistics(df)

In [ ]:
# path = orig_data_path
# files = glob.glob(orig_data_path+"*/*.zip",) # recursive=True
# for file in files:.query('value==823000000')
#     print(file)
#     # os.remove(file)

In [ ]:
fact_df[9:10]['element_id'].values

In [ ]:
fact_df[9:10]['element_id'].values

In [ ]:
fact_df[fact_df.Ja_item_name.str.contains('純利益')]

In [ ]:
fact_df.query("element_id=='ProfitLossAttributableToOwnersOfParentSummaryOfBusinessResults'")

In [ ]:
fact_df.query("element_id=='NetIncomeLossSummaryOfBusinessResults'")